# Decomposition for Solving Integer Programs
## Introduction
In this notebook, we will explore the idea of decomposition for solving integer programs (IPs). Integer programming is a general mathematical modeling tools for solving discrete optimization problems. It is widely used in industry, for example, [Amazon](https://www.truckstopsrouting.com/us/amazon-prime-same-day-deliveries/) relies on route optimization for planning deliveries, which can be modelled as IPs. [Sports scheduling](https://www.sciencedirect.com/science/article/abs/pii/S0305054812002869) is another area where IPs are widely used. The bad news is that solving general IPs are [NP-hard](https://en.wikipedia.org/wiki/Integer_programming). So solving an IP to optimality is often prohibitively expensive. In practice, general purpose IP solvers, such as Gurobi and SCIP, are used. These solvers implement the [branch-and-bound](https://en.wikipedia.org/wiki/Branch_and_bound) tree search algorithm together with a host of heuristics. Typically, these solvers are quite efficient at solving small scale problems with a small number of integer variables. However, once the number of integer variables becomes large, they become inefficient. A natural idea of mitigating this undesirable behavior is through the idea of decomposition. By breaking a large problem into a series of small problems, we can leverage existing strong solvers on the small problems. The main challenge is how to combine solutions generated from the small problems into a feasible solution to the original large problem.

## Setup
We will use the [python-mip](https://www.python-mip.com/) package with an open-source IP solver [CBC](https://projects.coin-or.org/Cbc). We study the problem of computing a [maximum cut (MaxCut)](https://en.wikipedia.org/wiki/Maximum_cut) of a weighted graph. The MaxCut problem is defined over a weighted graph $G = (V, E)$. The problem aims to divide the vertex set $V$ into two disjoint subsets $V = V_0 \cup V_1$ to maximize the total weights of cut edges, edges with one vertex in $V_0$ and the other in $V_1$. This problem can be formulated as IPs
>$\begin{align}
& \max \sum_{(i, j)\in E} w_{ij} e_{ij} \\
& s.t.  \\ 
& e_{ij} \le v_i + v_j, \forall (i, j) \in E \\
& e_{ij} + v_i + v_j \le 2, \forall (i, j) \in E \\
& v_i \in \{0, 1\}, \forall i \in V \\
& e_{ij} \in \{0, 1\}, \forall (i, j) \in E
\end{align}
$

Intuitively, values for $v_i$ indicate in which subset it is placed. The variables $e_{ij}$ associated with each edge have the value 1 if it is a cut edge and the value 0 otherwise. 

The following code block generates IPs based on MaxCut problems. It has two components, the **gen_graph** function generates a random graph, with the option to specify which distribution to draw a graph from. Next, the **create_opt** function takes input a graph and outputs an IP based on the formuation above.

In [0]:
!pip install mip
import mip
import networkx as nx
import numpy as np
import random


def gen_graph(max_n, min_n, g_type='barabasi_albert', edge=4):
    cur_n = np.random.randint(max_n - min_n + 1) + min_n
    if g_type == 'erdos_renyi':
        g = nx.erdos_renyi_graph(n = cur_n, p = 0.15)
    elif g_type == 'powerlaw':
        g = nx.powerlaw_cluster_graph(n = cur_n, m = 4, p = 0.05)
    elif g_type == 'barabasi_albert':
        g = nx.barabasi_albert_graph(n = cur_n, m = edge)
    elif g_type == 'watts_strogatz':
        g = nx.watts_strogatz_graph(n = cur_n, k = cur_n // 10, p = 0.1)

    for edge in nx.edges(g):
        g[edge[0]][edge[1]]['weight'] = random.uniform(0,1)

    return g


def getEdgeVar(m, v1, v2, vert):
    u1 = min(v1, v2)
    u2 = max(v1, v2)
    if not ((u1, u2) in vert):
        vert[(u1, u2)] = m.add_var(name='u%d_%d' %(u1, u2),
                                   var_type='B')

    return vert[(u1, u2)]


def getNodeVar(m, v, node):
    if not v in node:
        node[v] = m.add_var(name='v%d' %v,
                            var_type='B')

    return node[v]


def createOpt(G):
    m = mip.Model()
    # Emphasis is on finding good feasible solutions.
    m.emphasis = 1
    edgeVar = {}
    nodeVar = {}
    m.objective = 0
  
    for j, (v1, v2) in enumerate(G.edges()):
        e12 = getEdgeVar(m, v1, v2, edgeVar)
        node1 = getNodeVar(m, v1, nodeVar)
        node2 = getNodeVar(m, v2, nodeVar)

        m += e12 <= node1 + node2
        m += e12 + node1 + node2 <= 2
    
        m.objective = m.objective - (G[v1][v2]['weight']) * e12

    return m


def generateInstance(max_n, min_n, 
                     g_type='erdos_renyi', edge=4, outPrefix=None):
    G = gen_graph(max_n, min_n, g_type, edge)
    P = createOpt(G)

    return G, P

     |████████████████████████████████| 47.6MB 85kB/s 
Using Python-MIP package version 1.8.0


In [0]:
def generate_var_dict(model):
    '''Returns a dictionary mapping nodes in a graph to variables in a model.'''
    model_vars = model.vars
    num_vars = 0
    var_dict = {}

    for model_var in model_vars:
        if model_var.name.startswith('v'):
            num_vars += 1

    var_dict = dict([(i, ["v%d" %i]) for i in range(num_vars)])

    return var_dict


def uniform_random_clusters(var_dict, num_clusters):
    '''Return a random clustering. Each node is assigned to a cluster
    a equal probability.'''

    choices = list(range(num_clusters))
    clusters = dict([(i, []) for i in range(num_clusters)])

    for k in var_dict.keys():
        cluster_choice = random.choice(choices)
        clusters[cluster_choice].append(k)

    return clusters


def initialize_solution(var_dict, model):
    '''Initialize a feasible solution.

    Arguments:
      var_dict: a dict maps node index to node variable name.

    Returns:
      a dict maps node variable name to a value.
      a torch tensor view of the dict.
    '''

    sol = {}
    # the sol_vec needs to be of type float for later use with Pytorch.
    sol_vec = None
    init_obj = 0

    for k, var_list in var_dict.items():
        #sol_vec = np.zeros((len(var_dict), len(var_list)))
        for i, v in enumerate(var_list):
            sol[v] = 0      

    return sol, init_obj    

## Large Neighborhood Search (LNS)
We now describe the details of our LNS framework. At a high level, our LNS framework operates on an integer program (IP) via defining decompositions of its integer variables into disjoint subsets. Afterwards, we can select a subset and use an existing solver to optimize the variables in that subset while holding all other variables fixed. The benefit of this framework is that it is completely generic to any IP instantiation of any combinatorial optimization problem.

For an integer program $P$ with a set of integer variables $X$ (not necessarily all the integer variables), we define a decomposition of the set $X$ as a disjoint union $X_1 \cup X_2 \cup \cdots \cup X_k$. Assume we have an existing feasible solution $S_X$ to $P$, we view each subset $X_i$ of integer variables as a local neighborhood for search. We fix integers in $X\setminus X_i$ with their values in the current solution $S_X$ and optimize for variable in $X_i$ (referred as the $\texttt{FIX_AND_OPTIMIZE}$ function in Line 3 of Alg 1). As the resulting optimization is a smaller IP, we can use any off-the-shelf IP solver to carry out the local search. In our experiments, we use Gurobi to optimize the sub-IP. A new solution is obtained and we repeat the process with the remaining subsets.

<img src="https://drive.google.com/uc?export=view&id=1MJTIMV186ltfohVbZNPArxL9lEueRD4J" height="200"/>

In [0]:
import time

def LNS(model, num_clusters, steps, time_limit=3, verbose=False):
    """Perform large neighborhood search (LNS) on model. The descent order is random.

    Arguments:
      model: the integer program.
      num_clusters: number of clusters.
      steps: the number of decompositions to apply.
      var_dict: a dict maps node index to node variable name.
      time_limit: time limit for each LNS step.
      sol: (optional) initial solution.
      graph: networkx graph object.
      verbose: if True, print objective after every decomposition.
    """

    model.max_seconds = time_limit
    total_time = 0

    var_dict = generate_var_dict(model)

    start_time = time.time()
    sol, start_obj = initialize_solution(var_dict, model)

    cluster_list = []
    obj_list = [start_obj]

    for _ in range(steps):
        clusters = uniform_random_clusters(var_dict, num_clusters)

        sol, solver_time, obj = LNS_by_clusters(
            model.copy(), clusters, var_dict, sol)
        total_time += solver_time
        if verbose:
            print("objective: ", obj)

        cur_time = time.time()

        cluster_list.append(clusters)
        obj_list.append(obj)

    return total_time, obj, start_obj - obj, cluster_list, obj_list


def LNS_by_clusters(model, clusters, var_dict, sol):
    """Perform LNS by clusters. The order of clusters is from the largest to
       the smallest.

    Arguments:
      model: the integer program.
      clusters: a dict mapping cluster index to node index.
      var_dict: mapping node index to node variable name.
      sol: current solution.

    Returns:
      new solution. time spent by the solver. new objective.
    """

    # order clusters by size from largest to smallest
    sorted_clusters = sorted(clusters.items(), 
                             key=lambda x: len(x[1]), 
                             reverse=True)
    solver_t = 0

    for idx, cluster in sorted_clusters:
        sol, solver_time, obj = gradient_descent(
            model.copy(), cluster, var_dict, sol)
        solver_t += solver_time

    return sol, solver_t, obj


def gradient_descent(model, cluster, var_dict, sol):
    """Perform gradient descent on model along coordinates defined by 
       variables in cluster,  starting from a current solution sol.
    
    Arguments:
      model: the integer program.
      cluster: the coordinates to perform gradient descent on.
      var_dict: mapping node index to node variable name.
      sol: a dict representing the current solution.

    Returns:
      new_sol: the new solution.
      time: the time used to perform the descent.
      obj: the new objective value.
    """

    var_starts = []
    for k, var_list in var_dict.items():
        for v in var_list:
            # warm start variables in the current coordinate set with the existing solution.
            model_var = model.var_by_name(v)
            if k in cluster:
                var_starts.append((model_var, sol[v]))
            else:
                model += model_var == sol[v]

    # model.start = var_starts

    start_time = time.time()
    model.optimize()
    end_time = time.time()
    run_time = end_time - start_time
    new_sol = {}

    for k, var_list in var_dict.items():
        for v in var_list:
            var = model.var_by_name(v)
            try:
                new_sol[v] = round(var.x)
            except:
                return sol, run_time, -1

    return new_sol, run_time, model.objective_value

Now let's try this idea out with random decompositions. We first generate a random graph sampled according to the [Barabasi-Albert model](https://en.wikipedia.org/wiki/Barab%C3%A1si%E2%80%93Albert_model).

In [0]:
G, P = generateInstance(100, 100, g_type='barabasi_albert', edge=5)

For random decompositions, we randomly decompose the 100 nodes into 5 equally-sized subsets. Iteratively, we apply 10 decompositions in total and for each subproblem, we impose a time limit of 1 second.

In [0]:
start_time = time.time()
t, _, _, _, _ = LNS(P, 5, 10, time_limit=1, verbose=True)
end_time = time.time()
total_t = end_time - start_time
print('solver time: ', t)
print('total time: ', total_t)

objective:  -155.62469740679254
objective:  -165.36952461874057
objective:  -171.8718668480746
objective:  -173.53634467141407
objective:  -173.9082557153236
objective:  -173.9082557153237
objective:  -176.08994042301055
objective:  -176.0899404230104
objective:  -176.08994042301043
objective:  -176.0899404230105
solver time:  1.4998536109924316
total time:  3.400538682937622


To compare the solver's performance without decomposition, we give the solver 10 times the amount of wall-clock time to solve the problem and compare the final objective value.

In [0]:
status = P.optimize(max_seconds=total_t * 10)
print(status)
print(P.objective_value)

MIN
OptimizationStatus.FEASIBLE
1.5758919715881348
-162.95738511495105


As the results show, even though the solver uses substantially more time, it produces worse solution compared with the version with decomposition.